## Persistent Volume

In [1]:
%%writefile pv_dags.yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-dags
  labels:
    type: local
spec:
  storageClassName: microk8s-hostpath
  accessModes: ["ReadWriteMany"]
  capacity:
    storage: 20Gi
  hostPath:
    type: DirectoryOrCreate
    path: "/home/meng/Work/Airflow/dags"

Writing pv_dags.yaml


In [2]:
%%writefile pvc_dags.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-dags
  namespace: airflow
spec:
  storageClassName: microk8s-hostpath
  volumeName: airflow-dags
  accessModes: ["ReadWriteMany"]
  resources:
    requests:
      storage: 20Gi

Writing pvc_dags.yaml


## Dags

In [ ]:
%%writefile ~/work/Airflow/dags/tutorial.py
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

with DAG(
    dag_id="tutorial",
    start_date=datetime.datetime.today(),
    schedule_interval="15 * * * *"
) as dag:

    t1 = BashOperator(
        task_id="echo",
        bash_command="echo $PWD"
    )

    t1

## Images

In [ ]:
%%writefile ~/work/Airflow/requirements.txt
apache-airflow-providers-microsoft-mssql
apache-airflow-providers-mysql
apache-airflow-providers-google

In [ ]:
%%writefile ~/work/Airflow/Dockerfile
FROM apache/airflow:latest

USER root

ENV JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

RUN apt-get update -y \
    && apt-get install -y --no-install-recommends \
        build-essential \
        openjdk-11-jre-headless \
    && apt-get upgrade -y \
    && apt-get autoremove --purge -y \
    && apt-get autoclean -y \
    && rm -rf /var/lib/apt/lists/*

COPY ./dags/ /opt/airflow/dags/

USER airflow

COPY requirements.txt .

RUN python3 -m pip install --no-cache-dir -U pip wheel \
    && python3 -m pip install --no-cache-dir -Ur requirements.txt \
    && pip check \
    && rm -rf requirements.txt

In [ ]:
%%writefile ~/work/Airflow/Dockerfile
FROM apache/airflow:latest

COPY --chown=airflow:root ./dags/ /opt/airflow/dags/

In [ ]:
%%writefile ~/work/Airflow/build.sh
#!/bin/bash

export REPOSITORY=darklemon
export IMAGE=airflow
export TAG=latest

docker build . --no-cache --pull \
-f Dockerfile \
-t "${REPOSITORY}/${IMAGE}:${TAG}"

docker push "${REPOSITORY}/${IMAGE}:${TAG}"

In [ ]:
%%writefile ~/work/Airflow/build.sh
#!/bin/bash

export PROJECT="claritas-bigdata-poc"
export REGION="asia-southeast1"
export REPO="airflow"
export IMG="dags"
export TAG="0.0.1"
export TEMPLATE_IMG="${REGION}-docker.pkg.dev/${PROJECT}/${REPO}/${IMG}:${TAG}"

gcloud artifacts repositories create "${REPO}" \
--repository-format=Docker \
--project="${PROJECT}" \
--location="${REGION}"

gcloud builds submit . \
--tag="${TEMPLATE_IMG}"

## Install

In [3]:
%%writefile override.yaml
# Webserver secret key
webserverSecretKey: $(python3 -c 'import secrets; print(secrets.token_hex(16))')

# Executor
executor: CeleryKubernetesExecutor
logs:
  persistence:
    enabled: true
    
# Webserver-service
webserver:
  service:
    type: NodePort
    ports:
    - name: airflow-ui
      port: 8080

# dags
dags:
  persistence:
    enabled: true
    existingClaim: airflow-dags
        
# Images
# images:
#   airflow:
#     repository: darklemon/airflow
#     tag: latest
#     pullPolicy: Always

Overwriting override.yaml


In [4]:
%%writefile install.sh
#!/bin/bash

microk8s helm3 repo add apache-airflow https://airflow.apache.org
microk8s helm3 repo update

microk8s kubectl create namespace airflow
microk8s kubectl create -f pv_dags.yaml
microk8s kubectl create -f pvc_dags.yaml

microk8s helm3 upgrade \
--install airflow apache-airflow/airflow \
--namespace airflow \
-f override.yaml

Overwriting install.sh


## Uninstall

In [5]:
%%writefile uninstall.sh
#!/bin/bash

microk8s helm3 delete airflow \
--namespace airflow
microk8s helm3 repo rm apache-airflow
microk8s kubectl delete -f pv_dags.yaml
microk8s kubectl delete ns airflow

Overwriting uninstall.sh
